In [ ]:

import sys
import os
proj_root = os.path.abspath("..")  
if proj_root not in sys.path:
    sys.path.insert(0, proj_root)

from models.preprocessing import Preprocessor, Initial_Transformation  # noqa: E402


In [2]:
mobile , _ = Initial_Transformation(file_path="../models/dataset")
preprocessing = Preprocessor()

preprocessing.fit(mobile)
transformed = preprocessing.transform(mobile)

print("=" * 60)
print("FEATURE TRANSFORMATION RESULTS")
print("=" * 60)


print(transformed[50:60])
print(transformed.shape)

FEATURE TRANSFORMATION RESULTS
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000

In [6]:
from mlflow import MlflowClient
from pprint import pprint


In [11]:
client = MlflowClient(tracking_uri="http://127.0.0.1:8080")
all_experiments = client.search_experiments()

print(all_experiments)

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1760900447448, experiment_id='0', last_update_time=1760900447448, lifecycle_stage='active', name='Default', tags={}>]


In [12]:
default_experiment = [
    {"name": experiment.name, "lifecycle_stage": experiment.lifecycle_stage}
    for experiment in all_experiments
    if experiment.name == "Default"
][0]

pprint(default_experiment)

{'lifecycle_stage': 'active', 'name': 'Default'}
